In [1]:
# Imports
from utils import *

In [2]:
# Params
data_folder = "/home/savio/Documents/Tutorials/NLP_Tutorials/datasets/wmt-14-eng-deu"

In [2]:
# Download all the training, test and validation data
# 1. Training data - Combination of EuroParlv7, CommonCrawl and NewsCommentary (Total 4.5 million sentence pairs)
download_data(data_folder)



This may take a while.


Extracting training-parallel-europarl-v7.tgz...


Extracting training-parallel-commoncrawl.tgz...


Extracting training-parallel-nc-v9.tgz...






In [5]:
# Read in the training data and combine into a single dataset:
read_and_combine_data(data_folder)


Reading extracted files and combining...

Writing to single files...


In [6]:
# Perform byte-pair encoding and create a BPE model
perform_bpe(data_folder)


Learning BPE...

 BPE DONE!


Training parameters
  input: /home/savio/Documents/Tutorials/NLP_Tutorials/datasets/wmt-14-eng-deu/train.ende
  model: /home/savio/Documents/Tutorials/NLP_Tutorials/datasets/wmt-14-eng-deu/bpe.model
  vocab_size: 37000
  n_threads: 8
  character_coverage: 1
  pad: 0
  unk: 1
  bos: 2
  eos: 3

reading file...
learning bpe...
number of unique characters in the training data: 3762
number of deleted characters: 0
number of unique characters left: 3762
id: 4000=3785+8               freq: 606668      subword: ant=an+t
id: 5000=4691+3909            freq: 78525       subword: ▁know=▁kn+ow
id: 6000=3801+13              freq: 39092       subword: ash=as+h
id: 7000=3807+5197            freq: 25400       subword: usätz=us+ätz
id: 8000=3857+4012            freq: 18095       subword: ▁live=▁l+ive
id: 9000=4026+26              freq: 13882       subword: lav=la+v
id: 10000=5255+3903           freq: 11067       subword: ▁France=▁Fran+ce
id: 11000=8171+3849           freq: 9163        subword: ▁schaffe

In [3]:
# Perform filtering of the data:
perform_filtering(data_folder)


Loading BPE model...

Re-reading single files...

Filtering...


100%|██████████| 4520623/4520623 [02:48<00:00, 26874.38it/s]



Note: 13.85 per cent of en-de pairs were filtered out based on sub-word sequence length limits.

Re-writing filtered sentences to single files...

...FILTERING DONE!



In [3]:
# Initialize the data loaders: One for traing and one for validation 
from sequence_loader import SequenceLoader

source_suffix = "en"
target_suffix = "de"
tokens_in_batch = 2000

train_loader = SequenceLoader(data_folder, source_suffix, target_suffix, "train", tokens_in_batch)
val_loader   = SequenceLoader(data_folder, source_suffix, target_suffix, "val", tokens_in_batch)

In [4]:
# Define the model parameters:
d_model      = 512  # size of vectors throughout the transformer model
n_heads      = 8  # number of heads in the multi-head attention
d_queries    = 64  # size of query vectors (and also the size of the key vectors) in the multi-head attention
d_values     = 64  # size of value vectors in the multi-head attention
d_hidden     = 2048  # an intermediate size in the position-wise Feed forward networks
n_layers     = 6  # number of layers in the Encoder and Decoder
dropout_prob = 0.1  # dropout probability

In [5]:
import torch.backends.cudnn as cudnn

# Define the Training parameters;
batches_per_step = 25000 // tokens_in_batch  # perform a training step, i.e. update parameters, once every so many batches
print_frequency = 20  # print status once every so many steps
n_steps = 100000  # number of training steps
warmup_steps = 8000  # number of warmup steps where learning rate is increased linearly; twice the value in the paper, as in the official transformer repo.
step = 1  # the step number, start from 1 to prevent math error in the next line
lr = get_lr(step=step, d_model=d_model,
            warmup_steps=warmup_steps)  # see utils.py for learning rate schedule; twice the schedule in the paper, as in the official transformer repo.
start_epoch = 0  # start at this epoch
betas = (0.9, 0.98)  # beta coefficients in the Adam optimizer
epsilon = 1e-9  # epsilon term in the Adam optimizer
label_smoothing = 0.1  # label smoothing co-efficient in the Cross Entropy loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # CPU isn't really practical here
cudnn.benchmark = False  # since input tensor size is variable

In [6]:
from transformer import Transformer

# Create the positional encoding matrix:
positional_encoding = get_positional_encoding(d_model=d_model, max_length=160)  # positional encodings up to the maximum possible pad-length

# Instantiate the Transformer model
model = Transformer(vocab_size=train_loader.bpe_model.vocab_size(),
                    positional_encoding=positional_encoding,
                    num_layers= n_layers,
                    d_model=d_model,
                    n_heads=n_heads,
                    dropout_prob = dropout_prob,
                    d_hidden=d_hidden,
                    d_queries=d_queries,
                    d_values=d_values)

# Instantiate the Optimizer
optimizer = torch.optim.Adam(params=[p for p in model.parameters() if p.requires_grad],
                            lr=lr,
                            betas=betas,
                            eps=epsilon)

Model initialized.


In [7]:
from label_smoothed_ce import LabelSmoothedCE

# Setup the loss function: Label Smoother Cross Entropy

# Loss function
criterion = LabelSmoothedCE(eps=label_smoothing)

# Move to default device
model = model.to(device)
criterion = criterion.to(device)

# Find total epochs to train
epochs = (n_steps // (train_loader.n_batches // batches_per_step)) + 1

In [8]:
import time
from utils import *

def train(train_loader, model, criterion, optimizer, epoch, step):
    """
    One epoch's training.

    :param train_loader: loader for training data
    :param model: model
    :param criterion: label-smoothed cross-entropy loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    model.train()  # training mode enables dropout

    # Track some metrics
    data_time = AverageMeter()  # data loading time
    step_time = AverageMeter()  # forward prop. + back prop. time
    losses = AverageMeter()  # loss

    # Starting time
    start_data_time = time.time()
    start_step_time = time.time()

    # Batches
    for i, (source_sequences, target_sequences, source_sequence_lengths, target_sequence_lengths, src_padding_mask, target_padding_mask) in enumerate(
            train_loader):

        # Move to default device
        source_sequences = source_sequences.to(device)  # (N, max_source_sequence_pad_length_this_batch)
        target_sequences = target_sequences.to(device)  # (N, max_target_sequence_pad_length_this_batch)
        source_sequence_lengths = source_sequence_lengths.to(device)  # (N)
        target_sequence_lengths = target_sequence_lengths.to(device)  # (N)

        # Time taken to load data
        data_time.update(time.time() - start_data_time)

        src_padding_mask = src_padding_mask.to(device)
        target_padding_mask = target_padding_mask.to(device)

        # Forward prop.
        predicted_sequences = model(source_sequences, target_sequences, src_padding_mask,
                                    target_padding_mask)  # (1, target_sequence_length, vocab_size)

        # Note: If the target sequence is "<BOS> w1 w2 ... wN <EOS> <PAD> <PAD> <PAD> <PAD> ..."
        # we should consider only "w1 w2 ... wN <EOS>" as <BOS> is not predicted
        # Therefore, pads start after (length - 1) positions
        loss = criterion(inputs=predicted_sequences,
                         targets=target_sequences[:, 1:],
                         lengths=target_sequence_lengths - 1)  # scalar

        # Backward prop.
        (loss / batches_per_step).backward()

        # Keep track of losses
        losses.update(loss.item(), (target_sequence_lengths - 1).sum().item())

        # Update model (i.e. perform a training step) only after gradients are accumulated from batches_per_step batches
        if (i + 1) % batches_per_step == 0:
            optimizer.step()
            optimizer.zero_grad()

            # This step is now complete
            step += 1

            # Update learning rate after each step
            change_lr(optimizer, new_lr=get_lr(step=step, d_model=d_model, warmup_steps=warmup_steps))

            # Time taken for this training step
            step_time.update(time.time() - start_step_time)

            # Print status
            if step % print_frequency == 0:
                print('Epoch {0}/{1}-----'
                      'Batch {2}/{3}-----'
                      'Step {4}/{5}-----'
                      'Data Time {data_time.val:.3f} ({data_time.avg:.3f})-----'
                      'Step Time {step_time.val:.3f} ({step_time.avg:.3f})-----'
                      'Loss {losses.val:.4f} ({losses.avg:.4f})'.format(epoch + 1, epochs,
                                                                        i + 1, train_loader.n_batches,
                                                                        step, n_steps,
                                                                        step_time=step_time,
                                                                        data_time=data_time,
                                                                        losses=losses))

            # Reset step time
            start_step_time = time.time()

            # If this is the last one or two epochs, save checkpoints at regular intervals for averaging
            if epoch in [epochs - 1, epochs - 2] and step % 1500 == 0:  # 'epoch' is 0-indexed
                save_checkpoint(epoch, model, optimizer, prefix='step' + str(step) + "_")

        # Reset data time
        start_data_time = time.time()

In [9]:
from tqdm import tqdm

def validate(val_loader, model, criterion):
    """
    One epoch's validation.

    :param val_loader: loader for validation data
    :param model: model
    :param criterion: label-smoothed cross-entropy loss
    """
    model.eval()  # eval mode disables dropout

    # Prohibit gradient computation explicitly
    with torch.no_grad():
        losses = AverageMeter()
        # Batches
        for i, (source_sequence, target_sequence, source_sequence_length, target_sequence_length, src_padding_mask, target_padding_mask) in enumerate(
                tqdm(val_loader, total=val_loader.n_batches)):
            source_sequence = source_sequence.to(device)  # (1, source_sequence_length)
            target_sequence = target_sequence.to(device)  # (1, target_sequence_length)
            source_sequence_length = source_sequence_length.to(device)  # (1)
            target_sequence_length = target_sequence_length.to(device)  # (1)
            src_padding_mask = src_padding_mask.to(device)
            target_padding_mask = target_padding_mask.to(device)

            # Forward prop.
            predicted_sequence = model(source_sequence, target_sequence, src_padding_mask,
                                       target_padding_mask)  # (1, target_sequence_length, vocab_size)

            # Note: If the target sequence is "<BOS> w1 w2 ... wN <EOS> <PAD> <PAD> <PAD> <PAD> ..."
            # we should consider only "w1 w2 ... wN <EOS>" as <BOS> is not predicted
            # Therefore, pads start after (length - 1) positions
            loss = criterion(inputs=predicted_sequence,
                             targets=target_sequence[:, 1:],
                             lengths=target_sequence_length - 1)  # scalar

            # Keep track of losses
            losses.update(loss.item(), (target_sequence_length - 1).sum().item())

        print("\nValidation loss: %.3f\n\n" % losses.avg)

In [10]:
print(epochs)

21


In [11]:
## BEGIN THE TRAINING MY PADAWAN:
# Epochs
for epoch in range(start_epoch, epochs):
    # Step
    step = epoch * train_loader.n_batches // batches_per_step

    print(" Epoch # ", epoch)

    # One epoch's training
    train_loader.create_batches()
    train(train_loader=train_loader,
            model=model,
            criterion=criterion,
            optimizer=optimizer,
            epoch=epoch,
            step=step)

    # One epoch's validation
    val_loader.create_batches()
    validate(val_loader=val_loader,
                model=model,
                criterion=criterion)

    # Save checkpoint
    save_checkpoint(epoch, model, optimizer)

 Epoch #  0
Epoch 1/21-----Batch 240/59476-----Step 20/100000-----Data Time 0.003 (0.005)-----Step Time 3.124 (3.089)-----Loss 10.7014 (10.8921)
Epoch 1/21-----Batch 480/59476-----Step 40/100000-----Data Time 0.004 (0.005)-----Step Time 3.283 (3.121)-----Loss 10.2190 (10.6955)
Epoch 1/21-----Batch 720/59476-----Step 60/100000-----Data Time 0.003 (0.005)-----Step Time 3.417 (3.168)-----Loss 9.8823 (10.5131)
Epoch 1/21-----Batch 960/59476-----Step 80/100000-----Data Time 0.003 (0.005)-----Step Time 3.509 (3.217)-----Loss 9.7260 (10.3752)
Epoch 1/21-----Batch 1200/59476-----Step 100/100000-----Data Time 0.003 (0.005)-----Step Time 3.380 (3.246)-----Loss 9.8473 (10.2598)
Epoch 1/21-----Batch 1440/59476-----Step 120/100000-----Data Time 0.004 (0.005)-----Step Time 3.348 (3.274)-----Loss 9.9047 (10.1629)
Epoch 1/21-----Batch 1680/59476-----Step 140/100000-----Data Time 0.003 (0.005)-----Step Time 3.520 (3.300)-----Loss 9.3696 (10.0734)
Epoch 1/21-----Batch 1920/59476-----Step 160/100000-----

100%|██████████| 3000/3000 [00:37<00:00, 80.40it/s]



Validation loss: 3.921


 Epoch #  1
Epoch 2/21-----Batch 48/59476-----Step 4960/100000-----Data Time 0.003 (0.004)-----Step Time 2.964 (2.904)-----Loss 3.3859 (3.6551)
Epoch 2/21-----Batch 288/59476-----Step 4980/100000-----Data Time 0.004 (0.005)-----Step Time 3.063 (3.038)-----Loss 3.6989 (3.8092)
Epoch 2/21-----Batch 528/59476-----Step 5000/100000-----Data Time 0.005 (0.005)-----Step Time 3.110 (3.072)-----Loss 4.0231 (3.8118)
Epoch 2/21-----Batch 768/59476-----Step 5020/100000-----Data Time 0.003 (0.005)-----Step Time 2.962 (3.075)-----Loss 3.9093 (3.8073)
Epoch 2/21-----Batch 1008/59476-----Step 5040/100000-----Data Time 0.003 (0.005)-----Step Time 3.082 (3.078)-----Loss 3.7855 (3.8277)
Epoch 2/21-----Batch 1248/59476-----Step 5060/100000-----Data Time 0.008 (0.005)-----Step Time 3.107 (3.082)-----Loss 3.5333 (3.8450)
Epoch 2/21-----Batch 1488/59476-----Step 5080/100000-----Data Time 0.004 (0.005)-----Step Time 3.019 (3.084)-----Loss 3.2124 (3.8376)
Epoch 2/21-----Batch 1728/594

100%|██████████| 3000/3000 [00:37<00:00, 81.00it/s]



Validation loss: 3.471


 Epoch #  2
Epoch 3/21-----Batch 96/59476-----Step 9920/100000-----Data Time 0.003 (0.004)-----Step Time 3.049 (2.973)-----Loss 3.1500 (3.5123)
Epoch 3/21-----Batch 336/59476-----Step 9940/100000-----Data Time 0.003 (0.005)-----Step Time 3.064 (3.064)-----Loss 3.6707 (3.4468)
Epoch 3/21-----Batch 576/59476-----Step 9960/100000-----Data Time 0.004 (0.005)-----Step Time 3.195 (3.094)-----Loss 4.2061 (3.4413)
Epoch 3/21-----Batch 816/59476-----Step 9980/100000-----Data Time 0.004 (0.005)-----Step Time 3.086 (3.102)-----Loss 4.6861 (3.4418)
Epoch 3/21-----Batch 1056/59476-----Step 10000/100000-----Data Time 0.003 (0.005)-----Step Time 3.139 (3.108)-----Loss 2.8131 (3.4270)
Epoch 3/21-----Batch 1296/59476-----Step 10020/100000-----Data Time 0.003 (0.005)-----Step Time 3.130 (3.111)-----Loss 4.6388 (3.4281)
Epoch 3/21-----Batch 1536/59476-----Step 10040/100000-----Data Time 0.003 (0.005)-----Step Time 3.119 (3.114)-----Loss 4.1857 (3.4381)
Epoch 3/21-----Batch 1776/

...

100%|██████████| 3000/3000 [00:36<00:00, 81.72it/s]



Validation loss: 3.176


 Epoch #  4
Epoch 5/21-----Batch 180/59476-----Step 19840/100000-----Data Time 0.010 (0.005)-----Step Time 3.101 (3.013)-----Loss 3.9186 (3.1666)
Epoch 5/21-----Batch 420/59476-----Step 19860/100000-----Data Time 0.003 (0.005)-----Step Time 3.036 (3.050)-----Loss 2.9571 (3.1441)
Epoch 5/21-----Batch 660/59476-----Step 19880/100000-----Data Time 0.003 (0.005)-----Step Time 3.068 (3.075)-----Loss 2.7363 (3.1422)
Epoch 5/21-----Batch 900/59476-----Step 19900/100000-----Data Time 0.003 (0.005)-----Step Time 3.144 (3.084)-----Loss 3.0166 (3.1436)
Epoch 5/21-----Batch 1140/59476-----Step 19920/100000-----Data Time 0.004 (0.005)-----Step Time 3.085 (3.091)-----Loss 3.0148 (3.1489)
Epoch 5/21-----Batch 1380/59476-----Step 19940/100000-----Data Time 0.003 (0.005)-----Step Time 3.210 (3.096)-----Loss 4.3437 (3.1633)
Epoch 5/21-----Batch 1620/59476-----Step 19960/100000-----Data Time 0.003 (0.005)-----Step Time 3.031 (3.094)-----Loss 2.7845 (3.1577)
Epoch 5/21-----Batch 

100%|██████████| 3000/3000 [00:36<00:00, 82.02it/s]



Validation loss: 3.112


 Epoch #  5
Epoch 6/21-----Batch 228/59476-----Step 24800/100000-----Data Time 0.004 (0.005)-----Step Time 3.134 (3.029)-----Loss 2.9512 (3.0923)
Epoch 6/21-----Batch 468/59476-----Step 24820/100000-----Data Time 0.004 (0.005)-----Step Time 3.098 (3.058)-----Loss 3.3053 (3.0648)
Epoch 6/21-----Batch 708/59476-----Step 24840/100000-----Data Time 0.003 (0.005)-----Step Time 3.148 (3.071)-----Loss 2.3414 (3.0610)
Epoch 6/21-----Batch 948/59476-----Step 24860/100000-----Data Time 0.003 (0.005)-----Step Time 3.106 (3.076)-----Loss 3.2027 (3.0711)
Epoch 6/21-----Batch 1188/59476-----Step 24880/100000-----Data Time 0.004 (0.005)-----Step Time 3.160 (3.079)-----Loss 4.2910 (3.0719)
Epoch 6/21-----Batch 1428/59476-----Step 24900/100000-----Data Time 0.004 (0.005)-----Step Time 3.151 (3.085)-----Loss 3.7894 (3.0732)
Epoch 6/21-----Batch 1668/59476-----Step 24920/100000-----Data Time 0.003 (0.005)-----Step Time 3.025 (3.091)-----Loss 3.1544 (3.0738)
Epoch 6/21-----Batch 

KeyboardInterrupt: 

In [17]:
# Translate: 
from translate import translate

best, all = translate("Life is like riding a bicycle. To keep your balance, you must keep moving")

print(" Best translation: ", best)

 Best translation:  Das Leben ist wie ein Fahrrad, um Ihr Gleichgewicht zu bewahren, müssen Sie sich immer bewegen.


In [15]:
import torch
import sacrebleu
from translate import translate
from tqdm import tqdm
from sequence_loader import SequenceLoader
import youtokentome
import codecs
import os

# Use sacreBLEU in Python or in the command-line?
# Using in Python will use the test data downloaded in prepare_data.py
# Using in the command-line will use test data automatically downloaded by sacreBLEU...
# ...and will print a standard signature which represents the exact BLEU method used! (Important for others to be able to reproduce or compare!)
sacrebleu_in_python = True

data_folder = "/home/savio/Documents/Tutorials/NLP_Tutorials/datasets/wmt-14-eng-deu"

# Make sure the right model checkpoint is selected in translate.py

# Data loader
test_loader = SequenceLoader(data_folder=data_folder,
                             source_suffix="en",
                             target_suffix="de",
                             split="test",
                             tokens_in_batch=None)
test_loader.create_batches()

# Evaluate
with torch.no_grad():
    hypotheses = list()
    references = list()
    for i, (source_sequence, target_sequence, source_sequence_length, target_sequence_length, s_maks, t_mask) in enumerate(
            tqdm(test_loader, total=test_loader.n_batches)):
        hypotheses.append(translate(source_sequence=source_sequence,
                                    beam_size=4,
                                    length_norm_coefficient=0.6)[0])
        references.extend(test_loader.bpe_model.decode(target_sequence.tolist(), ignore_ids=[0, 2, 3]))
    if sacrebleu_in_python:
        print("\n13a tokenization, cased:\n")
        print(sacrebleu.corpus_bleu(hypotheses, [references]))
        print("\n13a tokenization, caseless:\n")
        print(sacrebleu.corpus_bleu(hypotheses, [references], lowercase=True))
        print("\nInternational tokenization, cased:\n")
        print(sacrebleu.corpus_bleu(hypotheses, [references], tokenize='intl'))
        print("\nInternational tokenization, caseless:\n")
        print(sacrebleu.corpus_bleu(hypotheses, [references], tokenize='intl', lowercase=True))
        print("\n")
    else:
        with codecs.open("translated_test.de", "w", encoding="utf-8") as f:
            f.write("\n".join(hypotheses))
        print("\n13a tokenization, cased:\n")
        os.system("cat translated_test.de | sacrebleu -t wmt14/full -l en-de")
        print("\n13a tokenization, caseless:\n")
        os.system("cat translated_test.de | sacrebleu -t wmt14/full -l en-de -lc")
        print("\nInternational tokenization, cased:\n")
        os.system("cat translated_test.de | sacrebleu -t wmt14/full -l en-de -tok intl")
        print("\nInternational tokenization, caseless:\n")
        os.system("cat translated_test.de | sacrebleu -t wmt14/full -l en-de -tok intl -lc")
        print("\n")
    print(
        "The first value (13a tokenization, cased) is how the BLEU score is officially calculated by WMT (mteval-v13a.pl). \nThis is probably not how it is calculated in the 'Attention Is All You Need' paper, however.\nSee https://github.com/tensorflow/tensor2tensor/issues/317#issuecomment-380970191 for more details.\n")

100%|██████████| 3003/3003 [12:03<00:00,  4.15it/s]



13a tokenization, cased:

BLEU = 22.45 54.3/28.1/16.5/10.1 (BP = 1.000 ratio = 1.028 hyp_len = 64440 ref_len = 62688)

13a tokenization, caseless:

BLEU = 22.89 55.3/28.6/16.8/10.3 (BP = 1.000 ratio = 1.028 hyp_len = 64440 ref_len = 62688)

International tokenization, cased:

BLEU = 23.11 55.1/28.8/17.1/10.5 (BP = 1.000 ratio = 1.018 hyp_len = 65829 ref_len = 64676)

International tokenization, caseless:

BLEU = 23.58 56.2/29.3/17.4/10.8 (BP = 1.000 ratio = 1.018 hyp_len = 65829 ref_len = 64676)


The first value (13a tokenization, cased) is how the BLEU score is officially calculated by WMT (mteval-v13a.pl). 
This is probably not how it is calculated in the 'Attention Is All You Need' paper, however.
See https://github.com/tensorflow/tensor2tensor/issues/317#issuecomment-380970191 for more details.

